## Task 
* Create a RAG pipeline that can take following text and answer following questions
1. Try different types of chunking to get better answers?At least 3
2. Does asking questions differently give better answers? Why?
3. Try a different similarity search instead of cosine similarity - do the answers improve?


In [1]:
sample_text = """
The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization. This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.

Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter. These tribes have unique languages, traditions, and knowledge of the ecosystem. However, many face threats from illegal land encroachment and industrial activities.

Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents. The Amazon River, which flows through the rainforest, is the second longest river in the world and carries more water than any other river.

Efforts to protect the Amazon include international agreements, conservation programs, and sustainable development projects that aim to balance economic growth with environmental protection. Many organizations and governments are working to reduce illegal logging and promote reforestation initiatives.
"""

In [2]:
questions_2 =[
    "What is the Amazon rainforest?",
    "Which countries does the Amazon span across?",
    "Why is deforestation a problem in the Amazon?",
    "How does the Amazon rainforest affect global weather patterns?",
    "What role do indigenous tribes play in the Amazon?",
    "What is the importance of the Amazon River?",
    "What types of wildlife can be found in the Amazon?",
    "How does deforestation contribute to climate change?",
    "What efforts are being made to protect the Amazon?",
    "Why is the Amazon considered a major carbon sink?"
]

In [3]:
#Import Libraries
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re

c:\Users\davor\miniconda3\envs\data_things\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Load Sentece Transformer Model
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

1. Try different types of chunking to get better answers?At least 3

In [5]:
#Functions to split text into paragraphs
def split_text_to_para(text):
    return [para.strip() for para in re.split("\n+", text) if para.strip()]

def split_text_to_sentece(text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]


In [6]:
from langchain. text_splitter import RecursiveCharacterTextSplitter
def sentece_based_chunking(text):
    fixed_overlap_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 300, # Number of cahracter per chunk
        chunk_overlap = 50,# Number of overlapping character between chunks
        separators=["\n\n", "\n", ".", " "]
    )

    #Split the text
    sentence_chunks = fixed_overlap_splitter.split_text(text)
    return sentence_chunks


In [7]:
#Store document embeddings
stored_texts_para = split_text_to_para(sample_text)
store_embeddings_para = model.encode(stored_texts_para, convert_to_numpy=True)

stored_texts_sente = split_text_to_sentece(sample_text)
stored_embeddings_sente = model.encode(stored_texts_sente, conver_to_numpy=True)

stored_texts_chunks = sentece_based_chunking(sample_text)
store_embeddings_chunks = model.encode(stored_texts_chunks, convert_to_numpy=True)

1. Sentence embeddings

In [8]:
from sklearn.metrics import euclidean_distances
#Functions to generate embeddings
def get_transformer_embeddings(texts):
    return model.encode(texts, convert_to_numpy=True)

#Function to retrieve relevant passage
def retrieve_passage(query):
    query_embedding = get_transformer_embeddings([query])
    similarities = cosine_similarity(query_embedding, stored_embeddings_sente)[0]
    best_match_idx = np.argmax(similarities)
    return stored_texts_sente[best_match_idx]

def retrieve_passage_2(query):
    query_embedding = get_transformer_embeddings([query])
    similarities = euclidean_distances(query_embedding, stored_embeddings_sente)[0]
    best_match_idx = np.argmin(similarities)
    return stored_texts_sente[best_match_idx]

#function to answer questions based on stored content
def answer_question(query):
    relevant_passage = retrieve_passage(query)
    return relevant_passage

def answer_question_2(query):
    relevant_passage = retrieve_passage_2(query)
    return relevant_passage

In [9]:
#Sample Q and A for cosine_similarity
print("\nSample Questions and answers:\n")
for question in questions_2:
    response = answer_question(question)
    print(f"Q: {question}\nA:{response}\n")


Sample Questions and answers:

Q: What is the Amazon rainforest?
A:The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers.

Q: Which countries does the Amazon span across?
A:It spans across nine countries, including Brazil, Peru, and Colombia.

Q: Why is deforestation a problem in the Amazon?
A:Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization.

Q: How does the Amazon rainforest affect global weather patterns?
A:Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents.

Q: What role do indigenous tribes play in the Amazon?
A:Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter.

Q: What is the impor

In [10]:
#Sample Q and A for euclidian distance
print("\nSample Questions and answers:\n")
for question in questions_2:
    response = answer_question_2(question)
    print(f"Q: {question}\nA:{response}\n")


Sample Questions and answers:

Q: What is the Amazon rainforest?
A:The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers.

Q: Which countries does the Amazon span across?
A:It spans across nine countries, including Brazil, Peru, and Colombia.

Q: Why is deforestation a problem in the Amazon?
A:Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization.

Q: How does the Amazon rainforest affect global weather patterns?
A:Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents.

Q: What role do indigenous tribes play in the Amazon?
A:Indigenous tribes have lived in the Amazon for thousands of years, relying on its rich biodiversity for food, medicine, and shelter.

Q: What is the impor

Based on the sentece chunking and tring cosine similarity and euclidian distances the answer does not have much difference on the sample question data. 

2. Paragraph embeddings

In [11]:
#Functions to generate embeddings
def get_transformer_embeddings(texts):
    return model.encode(texts, convert_to_numpy=True)

#Function to retrieve relevant passage
def retrieve_passage(query):
    query_embedding = get_transformer_embeddings([query])
    similarities = cosine_similarity(query_embedding, store_embeddings_para)[0]
    best_match_idx = np.argmax(similarities)
    return stored_texts_para[best_match_idx]

#function to answer questions based on stored content
def answer_question(query):
    relevant_passage = retrieve_passage(query)
    return relevant_passage

In [12]:
#Sample Q and A
print("\nSample Questions and answers:\n")
for question in questions_2:
    response = answer_question(question)
    print(f"Q: {question}\nA:{response}\n")


Sample Questions and answers:

Q: What is the Amazon rainforest?
A:The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Q: Which countries does the Amazon span across?
A:The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia. The rainforest is home to around 10% of the known species on Earth, including jaguars, sloths, and thousands of species of insects and birds.

Q: Why is deforestation a problem in the Amazon?
A:Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization. This

3.Fixed Size Embeddings

In [13]:
#Functions to generate embeddings
def get_transformer_embeddings(texts):
    return model.encode(texts, convert_to_numpy=True)

#Function to retrieve relevant passage
def retrieve_passage(query):
    query_embedding = get_transformer_embeddings([query])
    similarities = cosine_similarity(query_embedding, store_embeddings_chunks)[0]
    best_match_idx = np.argmax(similarities)
    return stored_texts_chunks[best_match_idx]

#function to answer questions based on stored content
def answer_question(query):
    relevant_passage = retrieve_passage(query)
    return relevant_passage

In [14]:
#Sample Q and A
print("\nSample Questions and answers:\n")
for question in questions_2:
    response = answer_question(question)
    print(f"Q: {question}\nA:{response}\n")


Sample Questions and answers:

Q: What is the Amazon rainforest?
A:The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia

Q: Which countries does the Amazon span across?
A:Efforts to protect the Amazon include international agreements, conservation programs, and sustainable development projects that aim to balance economic growth with environmental protection

Q: Why is deforestation a problem in the Amazon?
A:Deforestation is a significant threat to the Amazon, with thousands of square kilometers lost each year due to agriculture, logging, and urbanization

Q: How does the Amazon rainforest affect global weather patterns?
A:Scientists believe that the Amazon plays a crucial role in global weather patterns by releasing water vapor into the atmosphere, which influences rainfall across South America and even other continents

Q: What role do indigen

In [15]:
response = answer_question("What is the Amazon rainforest")
response

'The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia'

In [16]:
response = answer_question("Tell me about the rainforest of Amazon")
response

'The Amazon rainforest is the largest tropical rainforest in the world, covering approximately 5.5 million square kilometers. It spans across nine countries, including Brazil, Peru, and Colombia'

In [17]:
response = answer_question("What do you know about carbon sink and what about climate change")
response

'. This deforestation contributes to climate change, as the rainforest acts as a major carbon sink, absorbing millions of tons of carbon dioxide annually.'

Our Last model with the ssentece based chunking, prioritiazing separators does look like it gives much different answers. Though in general, rephrasing improve retrieval quality because embedings are based on how similar are two sentences.So using synonyms might help the model to find a better similarity question-answer wise.